In [ ]:
# Install dependencies
!pip install music21

# Import libraries
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import os

# Create output directories
os.makedirs("/kaggle/working/lstm_outputs/model_notes", exist_ok=True)
os.makedirs("/kaggle/working/lstm_outputs/weights", exist_ok=True)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train_model(model, network_input, network_output)

def get_notes():
    notes = []
    files = glob.glob("/kaggle/input/d/medamineharbaoui/midi-songs/midi_songs/*.mid")[:100]
    for file in files:
    #for file in glob.glob("/kaggle/input/d/medamineharbaoui/midi-songs/midi_songs/*.mid"):
        midi = converter.parse(file)
        print(f"Parsing {file}")
        notes_to_parse = None
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            duration = element.duration.quarterLength
            if duration < 0.75:
                duration_class = 'short'
            elif duration < 1.5:
                duration_class = 'medium'
            else:
                duration_class = 'long'
            if isinstance(element, note.Note):
                note_str = f"{str(element.pitch)}_{duration_class}"
                notes.append(note_str)
            elif isinstance(element, chord.Chord):
                chord_str = f"{'.'.join(str(n) for n in element.normalOrder)}_{duration_class}"
                notes.append(chord_str)
    with open('/kaggle/working/lstm_outputs/model_notes/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

def prepare_sequences(notes, n_vocab):
    sequence_length = 25
    note_names = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(note_names))
    network_input = []
    network_output = []
    for i in range(len(notes) - sequence_length):
        input_sequence = notes[i:i + sequence_length]
        output_sequence = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in input_sequence])
        network_output.append(note_to_int[output_sequence])
    n_patterns = len(network_input)
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output)
    return network_input, network_output

def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def train_model(model, network_input, network_output):
    filepath = "/kaggle/working/lstm_outputs/weights/weights-improve-LSTM-{epoch:02d}-{loss:.4f}.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(network_input, network_output, epochs=200, batch_size=32, callbacks=callbacks_list, verbose=1)

if __name__ == '__main__':
    train_network()

check best loss score

In [ ]:
import glob
import re
import zipfile
import os

# Directory containing .keras files
weights_dir = "/kaggle/working/lstm_outputs/weights/"

# Function to extract epoch and loss from filename
def parse_filename(filename):
    match = re.search(r"weights-improve-LSTM-(\d+)-(\d+\.\d{4})\.keras", filename)
    if match:
        epoch = int(match.group(1))
        loss = float(match.group(2))
        return epoch, loss
    return -1, float("inf")

# Get all .keras files and extract epoch/loss
keras_files = glob.glob(weights_dir + "*.keras")
files_with_loss = [(f, parse_filename(f)) for f in keras_files]
files_with_loss = [(f, epoch, loss) for f, (epoch, loss) in files_with_loss if epoch != -1]

# Find the file with the lowest loss
if files_with_loss:
    best_file, best_epoch, best_loss = min(files_with_loss, key=lambda x: x[2])
    print(f"Best file: {best_file}")
    print(f"Epoch: {best_epoch}, Loss: {best_loss}")

    # Create a zip file with the best file
    zip_path = "/kaggle/working/best_keras_weight.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(best_file, os.path.join("weights", os.path.basename(best_file)))
    print(f"Zip file created at: {zip_path}")
else:
    print("No valid .keras files found.")